In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.backends.cudnn as cudnn

import time
import os
import copy
import cv2

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
def train_model(model, criterion, optimizer, data_loaders, batch_size, num_epochs=30):
  start_time = time.time()

  train_acc_history = []
  val_acc_history = []

  best_model_over_epochs = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))

    # Train and validate over each epoch
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()  # Set model to training mode
      else:
        model.eval()   # Set model to evaluate mode

      batch_loss = 0.0
      batch_correct_preds = 0

      # Iterate over data
      for inputs, labels in tqdm(data_loaders[phase]):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward
        # track history if only in train
        with torch.set_grad_enabled(phase == 'train'):
          # Get model outputs and calculate loss
          outputs = model(inputs)
          loss = criterion(outputs, labels)

          _, preds = torch.max(outputs, 1)
          
          # Backward + Optimize only in training phase
          if phase == 'train':
            loss.backward()
            optimizer.step()
        
        batch_loss += loss.item() * inputs.size(0)
        batch_correct_preds += torch.sum(preds == labels.data)
      
      epoch_loss = batch_loss / len(data_loaders[phase].dataset)
      epoch_acc = batch_correct_preds.double() / len(data_loaders[phase].dataset)

      print('-' * 50)
      print('{} Loss: {:.4f}  Acc: {:.4f}\n\n'.format(phase, epoch_loss, epoch_acc))

      # Deep copy the model
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_over_epochs = copy.deepcopy(model.state_dict())
      
      if phase == 'val':
        val_acc_history.append(epoch_acc)
      else:
        train_acc_history.append(epoch_acc)

  time_elapsed = time.time() - start_time
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60,
                                                      time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  # Load best model weights
  model.load_state_dict(best_model_over_epochs)

  return model, train_acc_history, val_acc_history

In [4]:
def visualize_acc_hist(train_acc_history, val_acc_history):
  train_hist = [h.cpu().numpy() for h in train_acc_history]
  val_hist = [h.cpu().numpy() for h in val_acc_history]

  print('Training accuracy', np.around(np.array(train_hist), 3).reshape(len(train_hist),))
  print('Validation accuracy', np.around(np.array(val_hist), 3).reshape(len(val_hist),))

  plt.figure(figsize=(15, 9))
  plt.title("Accuracy over Number of Training Epochs", fontsize=15)
  plt.xlabel("Training Epochs", fontsize=14)
  plt.ylabel("Accuracy", fontsize=14)

  plt.plot(range(1, num_epochs+1), train_hist, label="Training", marker = 'o')
  plt.plot(range(1, num_epochs+1), val_hist, label="Validation", marker = 'o')

  plt.ylim((0,1.))
  plt.xticks(np.arange(1, num_epochs+1, 1.0))
  plt.legend(loc='lower left', fontsize=13)
  plt.show()

In [5]:
def show_statistics(y_test, y_pred, classNames):
  print("\n**** Classification Report ****")
  print(classification_report(y_test, y_pred, target_names=classNames))
  
def plot_confusion_matrix(y_test, y_pred, classNames):
    classes = len(classNames)
    cm = confusion_matrix(y_test, y_pred)

    con = np.zeros((classes, classes))
    for x in range(classes):
        for y in range(classes):
            con[x,y] = cm[x,y] / np.sum(cm[x,:])

    plt.figure(figsize=(15,15))
    sns.set(font_scale=1.0)
    sns.heatmap(con, annot=True, fmt='.2', cmap='Blues',
                xticklabels=classNames, yticklabels=classNames)
    plt.show()

In [6]:
batch_size = 32
input_size = 224

data_transforms = {
    'train': transforms.Compose([transforms.Resize(256),
                                 transforms.CenterCrop(input_size),
                                 transforms.ToTensor(),
                                 transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                      std=[0.229, 0.224, 0.225])
                                ]),
    'val': transforms.Compose([transforms.Resize(256),
                               transforms.CenterCrop(input_size),
                               transforms.ToTensor(),
                               transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                    std=[0.229, 0.224, 0.225])
                               ])
}

train_set = torchvision.datasets.FashionMNIST("./data",
                                              download=True,
                                              transform=data_transforms['train'])

val_set = torchvision.datasets.FashionMNIST("./data",
                                            download=True,
                                            train=False,
                                            transform=data_transforms['val'])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)

val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4)

data_loaders = {'train': train_loader,
                'val': val_loader}

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [7]:
num_epochs = 20
num_classes = len(train_set.classes)
feature_extract = True

In [8]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)

# Set all model parameters to not update
if feature_extract:
  for param in model.parameters():
    param.requires_grad = False

# Reinitialize layer’s parameters
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, num_classes)

model.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [10]:
model = model.to(device)
params_to_update = model.parameters()
print("Params to learn:")

if feature_extract:
  params_to_update = []
  for name, param in model.named_parameters():
    if param.requires_grad == True:
      params_to_update.append(param)
      print("\t",name)
else:
  for name, param in model.named_parameters():
    if param.requires_grad == True:
      print("\t",name)

# Observe that all parameters are being optimized
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 classifier.weight
	 classifier.bias


In [ ]:
criterion = nn.CrossEntropyLoss()
model, train_history, val_history = train_model(model, criterion, optimizer, data_loaders, batch_size, num_epochs)


Epoch 0/19


  0%|          | 0/1875 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 1875/1875 [3:51:25<00:00,  7.41s/it]


--------------------------------------------------
train Loss: 0.6083  Acc: 0.8012




100%|██████████| 313/313 [31:12<00:00,  5.98s/it]


--------------------------------------------------
val Loss: 0.4934  Acc: 0.8324


Epoch 1/19


 20%|█▉        | 370/1875 [47:37<3:15:13,  7.78s/it]

In [ ]:
visualize_acc_hist(train_history, val_history)


In [1]:
test_labels = []
predict_labels = []

for inputs, labels in tqdm(data_loaders['val']):
  test_labels.extend(list(labels.detach().cpu().numpy()))

  inputs = inputs.to(device)
  labels = labels.to(device)

  predicted = model(inputs)
  _, preds = torch.max(predicted, 1)
  predict_labels.extend(list(preds.detach().cpu().numpy()))

print(test_labels)
print(predict_labels)

NameError: ignored

In [ ]:
show_statistics(test_labels, predict_labels, list(train_set.classes))


In [ ]:
batch = next(iter(val_loader))
images, labels = batch
print(type(images), type(labels))
print(images.shape, labels.shape)

In [ ]:
grid = torchvision.utils.make_grid(images[:10], nrow=10)

plt.figure(figsize=(15, 20))
plt.imshow(np.transpose(grid, (1, 2, 0)))
print("Labels: ", end=" ")
for i, label in enumerate(labels[:15]):
  print(val_set.classes[label], end=", ")

print("\nPredicted: ", end=" ")
for i, label in enumerate(predict_labels[:15]):
  print(val_set.classes[label], end=", ")

In [ ]:
class SiameseFashionMNIST(torchvision.datasets.FashionMNIST):
  def __init__(self, *args, **kwargs):
    super(SiameseFashionMNIST, self).__init__(*args, **kwargs)
    if kwargs["train"] is True:
      self.data, self.labels = self.train_data, self.train_labels
    else:
      self.data, self.labels = self.test_data, self.test_labels

  def __getitem__(self, idx):
    x1, t1 = self.data[idx], self.labels[idx]

    is_diff = random.randint(0, 1)
    while True:
      idx2 = random.randint(0, len(self)-1)
      x2, t2 = self.data[idx2], self.labels[idx2]
      if is_diff and t1 != t2:
        break
      if not is_diff and t1 == t2:
        break

    x1, x2 = Image.fromarray(x1.numpy()), Image.fromarray(x2.numpy())
    if self.transform is not None:
      x1, x2 = self.transform(x1), self.transform(x2)
    return x1, x2, int(is_diff), t1


siamese_data_transforms = transforms.Compose([transforms.ToTensor()])

siamese_train_loader = torch.utils.data.DataLoader(
    SiameseFashionMNIST("./data",
                        download=True,
                        train=True,
                        transform=siamese_data_transforms),
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4)

siamese_test_loader = torch.utils.data.DataLoader(
    SiameseFashionMNIST("./data",
                        download=True,
                        train=False,
                        transform=siamese_data_transforms),
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=4)

In [ ]:
class Siamese(nn.Module):
  def __init__(self):
    super(Siamese, self).__init__()
    self.c1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
    self.bn1 = nn.BatchNorm2d(32)
    self.c2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.bn2 = nn.BatchNorm2d(64)
    self.c3 = nn.Conv2d(64, 128, kernel_size=3, padding=0)
    self.bn3 = nn.BatchNorm2d(128)

    self.fc4 = nn.Linear(128, 64)
    self.bn4 = nn.BatchNorm1d(64)
    self.fc5 = nn.Linear(64, 2)

  def forward(self, x):
    h = F.max_pool2d(self.bn1(self.c1(x)), 2)
    h = F.max_pool2d(self.bn2(self.c2(h)), 2)
    h = F.avg_pool2d(self.bn3(self.c3(h)), 5)

    h = self.bn4(self.fc4(h.view(h.size(0), -1)))
    return self.fc5(h)


def contractive_loss(o1, o2, y):
  g, margin = F.pairwise_distance(o1, o2), 5.0
  loss = (1 - y) * (g ** 2) + y * (torch.clamp(margin - g, min=0) ** 2)
  return torch.mean(loss)

In [ ]:
siamese_model = Siamese().cuda()

opt = optim.SGD(siamese_model.parameters(), lr=0.01, momentum=0.9)

scheduler = optim.lr_scheduler.MultiStepLR(opt, [5, 10], 0.1)
cudnn.benckmark = True

best_siamese_model_over_epochs = copy.deepcopy(siamese_model.state_dict())
best_loss = np.Inf

for e in range(20):
  scheduler.step()
  model.train()
  train_loss, train_n = 0, 0
  for x1, x2, y, label in tqdm(siamese_train_loader, total=len(siamese_train_loader), leave=False):
  # for x1, x2, y in siamese_train_loader:
    x1, x2 = Variable(x1.cuda()), Variable(x2.cuda())
    y = Variable(y.float().cuda()).view(y.size(0), 1)

    o1, o2 = siamese_model(x1), siamese_model(x2)
    loss = contractive_loss(o1, o2, y)
    opt.zero_grad()
    loss.backward()
    opt.step()
    train_loss = loss.detach().cpu().numpy() * y.size(0)
    train_n += y.size(0)

  siamese_model.eval()
  test_loss, test_n = 0, 0
  for x1, x2, y, label in tqdm(siamese_test_loader, total=len(siamese_test_loader), leave=False):
    x1, x2 = Variable(x1.cuda()), Variable(x2.cuda())
    y = Variable(y.float().cuda()).view(y.size(0), 1)

    o1, o2 = siamese_model(x1), siamese_model(x2)
    loss = contractive_loss(o1, o2, y)
    test_loss = loss.detach().cpu().numpy() * y.size(0)
    test_n += y.size(0)
  
  if best_loss > test_loss:
    best_loss = test_loss
    best_siamese_model_over_epochs = copy.deepcopy(siamese_model.state_dict())
    
  print("\t".join(["Epoch", "TrainLoss", "TestLoss"]))
  print("{}\t{:.6f}\t{:.6f}".format(e, train_loss / train_n, test_loss / test_n))

# Load best model weights
siamese_model.load_state_dict(best_siamese_model_over_epochs)

In [ ]:
siamese_model.eval()

inputs, embs, targets, labels = [], [], [], []
for x, x2, t, label in tqdm(siamese_test_loader, total=len(siamese_test_loader)):
# for x, x2, t in siamese_test_loader:
  x = Variable(x.cuda())
  o1 = siamese_model(x)
  inputs.extend(list(x.cpu().data.numpy()))
  embs.extend(list(o1.cpu().data.numpy()))
  targets.extend(list(t.numpy()))
  labels.extend(list(label.numpy()))

In [ ]:
val_set.classes
obj_categories = val_set.classes

colors = plt.cm.rainbow(np.linspace(0, 1, len(obj_categories)))
plt.figure(figsize=(10, 10))

embs = np.array(embs)
labels = np.array(labels)

for c_group, (c_color, c_label) in enumerate(zip(colors, obj_categories)):
  plt.scatter(embs[np.where(labels == c_group), 0].flatten()[:50],
              embs[np.where(labels == c_group), 1].flatten()[:50],
              marker='o',
              color=c_color,
              linewidth='1',
              alpha=0.8,
              label=c_label)

plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('t-SNE on Testing Samples')
plt.legend(loc='best')
plt.savefig('clothes-dist.png')
plt.show(block=False)